<a href="https://colab.research.google.com/github/aruaru0/kaggle_h_and_m/blob/main/H%26M_item2vec_ver2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## item2vecの辞書を作成（各列を１つの文章に対応させたバージョン）

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import defaultdict
import json
import pickle
from gensim.models import word2vec
import multiprocessing
cpu = multiprocessing.cpu_count()

## データ読み込み

In [3]:
transactions = pd.read_csv("/content/drive/MyDrive/datas/h_and_m/transactions_train.csv",dtype=str, encoding='utf8')
articles = pd.read_csv("/content/drive/MyDrive/datas/h_and_m/articles.csv",dtype=str, encoding='utf8')
customers = pd.read_csv("/content/drive/MyDrive/datas/h_and_m/customers.csv",dtype=str, encoding='utf8')

## テキストを作成（transactions）

In [4]:
text = []

In [6]:
for i in tqdm(range(len(transactions))) :
  t_dat, customer_id, article_id, _, _  = transactions.iloc[i]
  s = t_dat[:-3] + " " + customer_id + " " + article_id
  text.append(s)

len(text)

100%|██████████| 31788324/31788324 [34:41<00:00, 15269.83it/s]


31788324

In [20]:
with open('transactions.pkl', 'wb') as f :
  pickle.dump(text,f)

## テキストを作成（cutomers)

In [18]:
ctext = []
post = dict()
num = 0
for i in tqdm(range(len(customers))) :
  customer_id, _, _, _, _, age, postal_code = customers.iloc[i]
  if postal_code in post :
    pcode = str(post[postal_code])
  else :
    pcode = str(num)
    num += 1
  s = customer_id + " age" + str(age) + " post" + pcode
  ctext.append(s)

len(ctext), ctext[:10]

100%|██████████| 1371980/1371980 [01:30<00:00, 15097.65it/s]


(1371980,
 ['00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657 age49 post0',
  '0000423b00ade91418cceaf3b26c6af3dd342b51fd051eec9c12fb36984420fa age25 post1',
  '000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318 age24 post2',
  '00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2c5feb1ca5dff07c43e age54 post3',
  '00006413d8573cd20ed7128e53b7b13819fe5cfc2d801fe7fc0f26dd8d65a85a age52 post4',
  '000064249685c11552da43ef22a5030f35a147f723d5b02ddd9fd22452b1f5a6 agenan post5',
  '0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d8cd0c725276a467a2a age20 post6',
  '00007d2de826758b65a93dd24ce629ed66842531df6699338c5570910a014cc2 age32 post7',
  '00007e8d4e54114b5b2a9b51586325a8d0fa74ea23ef77334eaec4ffccd7ebcc age20 post8',
  '00008469a21b50b3d147c97135e25b4201a8c58997f78782a0cc706645e14493 age20 post9'])

In [19]:
with open('customers.pkl', 'wb') as f :
  pickle.dump(ctext,f)

## テキストを保存（articles）

In [27]:
atext = []
for i in tqdm(range(len(articles))) :
  x = articles.iloc[i]
  article_id = x['article_id']
  product_code = x['product_code']
  product_type_no = x['product_type_no']
  graphical_appearance_no = x['graphical_appearance_no']
  colour_group_code = x['colour_group_code']
  s = article_id + " pc" + product_code + " pt" + product_type_no + " gan" + graphical_appearance_no + " cg"+colour_group_code
  atext.append(s)

len(atext), atext[:10]

100%|██████████| 105542/105542 [00:09<00:00, 11608.64it/s]


(105542,
 ['0108775015 pc0108775 pt253 gan1010016 cg09',
  '0108775044 pc0108775 pt253 gan1010016 cg10',
  '0108775051 pc0108775 pt253 gan1010017 cg11',
  '0110065001 pc0110065 pt306 gan1010016 cg09',
  '0110065002 pc0110065 pt306 gan1010016 cg10',
  '0110065011 pc0110065 pt306 gan1010016 cg12',
  '0111565001 pc0111565 pt304 gan1010016 cg09',
  '0111565003 pc0111565 pt302 gan1010016 cg13',
  '0111586001 pc0111586 pt273 gan1010016 cg09',
  '0111593001 pc0111593 pt304 gan1010016 cg09'])

In [28]:
with open('articles.pkl', 'wb') as f :
  pickle.dump(atext,f)

## テキストファイルに保存

In [29]:
all = text + ctext + atext 
len(all)

33265846

In [31]:
# テキスト形式にしｔ保存
with open('items.txt', 'w') as f:
    for e in tqdm(all) :
      f.write(e + '\n')

100%|██████████| 33265846/33265846 [00:25<00:00, 1310556.91it/s]


In [32]:
!cp *.pkl *.txt /content/drive/MyDrive/datas/h_and_m/newItem2Vec

## item2vec辞書作成

In [34]:
sentences = word2vec.LineSentence('items.txt')

# word2vecのモデルを構築
#     sg: 1ならskip-gram(cbowではなく)
#     size: 単語ベクトルの次元数
#     window: ウィンドウサイズ(今回は1行に含まれる全ての単語を対象とするため大きな値を設定)
#     hs: 0ならネガティブサンプリング(1なら階層的ソフトマックス)
#     negative: ネガティブサンプリングする単語数(結果への影響が予測できなかったのでデフォルトの5に設定)
#     seed: 乱数シード(結果を安定させたいので固定値を設定)
#model = word2vec.Word2Vec(sentences, sg=1, size=100, window=100000, hs=0, negative=5, seed=42)
model = word2vec.Word2Vec(sentences,  size=128, window=100000, seed=42, min_count=1, workers=cpu)

In [35]:
model.save("/content/drive/MyDrive/datas/h_and_m/newItem2Vec/word2vec.model")

In [36]:
id = '000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318'
model.wv.most_similar(id, topn=5)

[('5de4e583177a20b756360af9851ec968375c44ab3ee20ebef855c0f446ba961d',
  0.9895027875900269),
 ('3bcd3076d4a83d2397414cdf9f66da7cf05054ef5fdaee2b4f8da2760529d894',
  0.9894407391548157),
 ('01a2c06f2da389245e326eeac70cca42f870bba79a9fef2e5fef21f27bcf757a',
  0.9887876510620117),
 ('83e116d976f1ab290bf87a2b862bd206cb694d8dfeae062a81631b76949de27c',
  0.9884083271026611),
 ('0d29edfbaff1c3e8b926135a72565d80461b7891b6b8beb7005bd197c3e32149',
  0.9880269169807434)]

In [45]:
l = ['0541518023', '0663713001', '0685687001', '0554757003']
for e in l:
  res = model.wv.similarity(id, e)
  print(res)

0.352682
0.32407683
0.23754314
0.3784091
0.13633782


In [60]:
print(model.wv.similarity('age20', 'age25'))
print(model.wv.similarity('age25', 'age52'))
model.wv.most_similar('age25', topn=20)

0.41733956
0.098321795


[('age24', 0.7840039730072021),
 ('age26', 0.7803231477737427),
 ('age23', 0.7514244318008423),
 ('age27', 0.7355345487594604),
 ('age28', 0.6681232452392578),
 ('age22', 0.6419914364814758),
 ('age29', 0.539645791053772),
 ('post1325686', 0.5327457189559937),
 ('post932006', 0.5103806257247925),
 ('post1353757', 0.5046486854553223),
 ('post1132238', 0.5004220604896545),
 ('post1228311', 0.4994319975376129),
 ('post1095593', 0.4977501630783081),
 ('post1369919', 0.49208348989486694),
 ('post1354543', 0.49184343218803406),
 ('post1363857', 0.491605669260025),
 ('post951146', 0.4914271831512451),
 ('post1122536', 0.4913509488105774),
 ('post1231404', 0.48656782507896423),
 ('post1311484', 0.48302674293518066)]